In [ ]:
⎕IO ← 1            ⍝ Index origin is ONE
]box on -style=max ⍝ Show boxes at max verbosity
]rows on           ⍝ Don't wrap long output lines
I←⌷⍨∘⊃⍨⍤0 99       ⍝ Sane indexing
assert ← {⍺ ← 'assertion failure' ⋄ 0∊⍵: ⍺ ⎕signal 8 ⋄ shy ← 0} ⍝ assert

## Part 1 & 2: moving crates

#### Example data

In [ ]:
starting moves ← {⍵⊆⍨×≢¨⍵}⊃⎕NGET'ex1.txt'1      ⍝ partition by the blank line
starting

In [ ]:
↑starting                                       ⍝ mix
⍉↑starting                                      ⍝ transpose
↓⍉↑starting                                     ⍝ split
{4|⍳≢⍵}↓⍉↑starting                              ⍝ count up in fours
{2=4|⍳≢⍵}↓⍉↑starting                            ⍝ select the "2s"
{⍵/⍨2=4|⍳≢⍵}↓⍉↑starting                         ⍝ partition against the input
⎕ ← starting ← ~∘' '¨{⍵/⍨2=4|⍳≢⍵}↓⍉↑starting    ⍝ remove the whitespace

In [ ]:
⎕ ← moves ← ⍎¨¨'\d+'⎕S'&'¨moves                     ⍝ pull out the numbers ignore the rest

### pop_and_move explained:
```
⍝ ⍺ is a 2-vector: (number_to_lift, 0).
⍝ ⍵ is the two relevant stacks (from and to).
⍝ ⍺⍺ is the function to apply to the lifted items (rotate for part 1, null op for part 2)
pop_and_move ← {
    t←⍺↓¨⍵               ⍝ (1)
    u←⍺⍺¨⌽⍺↑¨⍵           ⍝ (2)
    u,¨t                 ⍝ (3)
}
```
(1) drops the specified number of items from the first stack (and 0 from the other)

(2) takes the specified number of items from the first stack (and 0 from the other), swaps them and then applies the ⍺⍺ function to them

(3) joins the results from (1) and (2) together

In [ ]:
pop_and_move ← {t←⍺↓¨⍵ ⋄ u←⍺⍺¨⌽⍺↑¨⍵ ⋄ u,¨t}
⍝ example in steps
t←(2 0) {⍺↓¨⍵} 'DCM2' 'NZ1'                    ⍝ (1) => ['M2', 'NZ1']
u←(2 0) ⌽ {⍺⍺¨⌽⍺↑¨⍵} 'DCM2' 'NZ1'              ⍝ (2) => ['', 'CD']
u,¨t                                           ⍝ (3)

In [ ]:
update←{(⊃⍺)0⍺⍺ pop_and_move @(1↓⍺)⊢⍵}

### update explained:
```
update←{(⊃⍺)0⍺⍺ pop_and_move @(1↓⍺)⊢⍵}
```
⍺ is the move, e.g. (1 2 1); ⍵ is the current state of the crates, e.g. ('NZ1' 'DCM2 'P3'); and, ⍺⍺ is the function to pass to pop_and_move.

How @ works: `{5+⍵}@(1 3)⊢1 2 3 4 5` adds 5 to the values at index 1 and 3.

Note: ⊢ stops us from having to put parentheses around the whole @ term, i.e. avoids `({5+⍵}@(1 3))1 2 3 4 5`

So, `@(1↓⍺)⊢⍵` sets the indices to replace in ⍵ to the 'from' and 'to' (drops the 'number to lift').

`(⊃⍺)0` gives a 2-vector of (number_to_lift, 0) which is passed to pop_and_move.

To summarise, update replaces the changed stacks with the result of pop_and_move. 


In [ ]:
⍝ alpha is the current move, reduce goes through them in reverse order using the last item as the starting value (see below)
assert 'CMZ' = ⊃¨⊃⌽update/(⌽moves),⊂starting ⍝ part 1
assert 'MCD' = ⊃¨⊃⊢update/(⌽moves),⊂starting ⍝ part 2

### How reduce behaves
Consider this example:
```
+/1 2 3 4
```
Reduce sets the initial value to 4, and then adds the 3, then the two, and then the one, i.e. it works right to left.

⍺ is the value being added.

⍵ is the running total.


In [ ]:
add ← {⎕ ← ⍺ ⍵ ⋄ ⍺ + ⍵}
add/1 2 3 4

#### Real data

In [ ]:
starting moves ← {⍵⊆⍨×≢¨⍵}⊃⎕NGET'input.txt'1
starting ← ~∘' '¨{⍵/⍨2=4|⍳≢⍵}↓⍉↑starting
moves ← ⍎¨¨'\d+'⎕S'&'¨moves
assert 'LBLVVTVLP' = ⊃¨⊃⌽update/(⌽moves),⊂starting ⍝ part 1
assert 'TPFFBDRJD' = ⊃¨⊃⊢update/(⌽moves),⊂starting ⍝ part 2